In [4]:
import numpy as np
from math import log2
import time

In [5]:
#===========================================#
#to use for testing, amend path to txt file
#===========================================#

clean_data = np.loadtxt("clean_dataset.txt") 
noisy_data = np.loadtxt("noisy_dataset.txt")

In [6]:
def entropy(roomData):
    H=0.0
    i=0
    count = {1:0, 2:0, 3:0, 4:0} #init dictionary to count room instances in dataset
    size = len(roomData)
    while i<size:
        if roomData[i][-1] == 1:
            count[1]+=1
        elif roomData[i][-1] == 2:
            count[2]+=1
        elif roomData[i][-1] == 3:
            count[3]+=1
        elif roomData[i][-1] == 4:
            count[4]+=1
        i+=1;
    for i in (1,2,3,4):
        if count[i] > 0:
            H += ((-count[i])/size)*(log2(count[i]/size))
    return H

In [7]:
def InfoGain(All, Left, Right):
    
    H_SAll=entropy(All) #entropy of parent datset
    Total=len(Left)+len(Right)
    Remainder=((len(Left)/Total)*entropy(Left))+((len(Right)/Total)*entropy(Right)) #weighted entropies of left and right datasets
    Gains=H_SAll-Remainder
    
    return Gains

In [8]:
def FIND_SPLIT(dataset):
    emitter = 0
    value = 0
    max_info_gain = 0
    for x in range(len(dataset[0])-1): #first 7 entries, exclude room
        
        ds = np.array(sorted(dataset, key = lambda y: y[x], reverse=True)) #sort n dimensional array by emitter
        
        for r in range(len(ds)): #iterate over every value for each emitter
            
            split_point = ds[r][x] #current point to check
            if(r!=len(ds)-1 and ds[r+1][x] == split_point): #check for repeated values, use last instance of value as split point
                continue
            info_gain = InfoGain(ds, ds[:r+1],ds[r+1:]) #calculte information gain for current point
            
            if(info_gain > max_info_gain): #if information gain is higher, update the best split point and emitter
                emitter = x
                value = split_point
                max_info_gain = info_gain
            
            
        
    return emitter, value

In [9]:
def split(clean_rows):
    Left = []
    Right = []
    em, val = FIND_SPLIT(clean_rows) #return the emitter and value for best split
    
    for i in range(len(clean_rows)): #iterate over dataset, using best split point, seperate dataset into left and right
        if(clean_rows[i][em] >= val):  
            Left.append(clean_rows[i])
        
        else:
            Right.append(clean_rows[i])
        
    
    
    return np.array(Left),np.array(Right),em,val

In [10]:
def tree_learning(training_data, depth):
    
    if(entropy(training_data) == 0 and len(training_data) != 0): #if all room labels are the same, return leaf which contains room number
        count = len(training_data) #counting the number of entries in the dataset where the room labels are all the same (used later for pruning purposes)
        label = training_data[0][-1]
        leaf = {'emitter':-1, 'value':-1, 'room': label, 'right':-1, 'left':-1, 'count': count}
        return leaf, depth
    
    else: 
        ldata, rdata, em, val = split(training_data) #split data to get the child datasets and split point
        root = {'emitter': em, 'value': val, 'room': -1, 'right':-1, 'left':-1, 'count': -1} #create non leaf node
        root['left'], l_depth = tree_learning(ldata, depth+1) #recursively call tree learning to generate rest of tree, increasing depth
        root['right'], r_depth = tree_learning(rdata, depth+1)
        return root, max(l_depth, r_depth) #return root of tree and depth

In [11]:
#===========================================#
#for testing tree generation only
#===========================================#
tree_root, depth = tree_learning(noisy_data, 0)
#print(tree_root)

In [12]:
def return_result(row, root):
    
    if(root['room']!= -1): #check for leaf node, return room
        return root['room']
    else:
        if(row[root['emitter']]>=root['value']): #traverse tree to find leaf node
            return return_result(row, root['left'])
        else:
            return return_result(row, root['right'])

In [13]:
def confusion_matrix(results_table): 
    
    cm = np.zeros((4,4)) #empty confusion matrix
    
    for i in range (1,5):
        for pair in results_table: #pair[0] is actual value, pair[1] is predicted value by decision tree
            if(i == pair[0] and pair[0] == pair[1]): #count correct predictions and update confusion matrix
                    cm[i-1][i-1]+=1
            if(i == pair[0] and pair[1] != i):
                    cm[i-1][int(pair[1])-1]+=1
    return cm

In [14]:
def accuracy(cm):
    if(np.sum(cm) == 0):
        return 0
    return np.trace(cm)/np.sum(cm) #returns accuracy from confusion matrix

In [15]:
#===========================================#
#function returns BOTH confusion matrix and accuracy, different from specification which only required us to return accuracy
#===========================================#

def evaluate(test, root): 
    
    results_table = []
    
    for row in test: #iterate over test data and generate prediction
        prediction = return_result(row, root)
        results_table.append([row[-1],prediction]) #keep track of actual outcome vs predicted outcome
        
    cm = confusion_matrix(results_table) #generate confusion matrix from results table
    tree_accuracy = accuracy(cm) #calculate accuracy from confusion matrix
    
    return cm, tree_accuracy 

In [16]:
#===========================================#
#cross validation WITHOUT pruning
#===========================================#

def cross_validation(dataset):

    np.random.shuffle(dataset) #shuffles dataset

    fold_size = int(len(dataset)/10) 
    running_cm = np.zeros((4,4)) #generate empty confusion matrix
    
    for i in range(10):
        
        #cycling test and training data for cross valdiation
        test = dataset[0+fold_size*i:fold_size*(i+1)] 
        training = np.vstack((dataset[0:0+fold_size*i],dataset[fold_size*(i+1):])) 
        
        root, depth = tree_learning(training, 0) #generate tree for each cycled training dataset
        
        confusion_matrix, tree_accuracy = evaluate(test, root) #generate confusion matrix for current tree   
        running_cm += confusion_matrix #keep track of confusion matrices to calculate average
            
    avg_cm = running_cm/10 #calculate average confusion matrix
    
    return accuracy(avg_cm) #return average accuracy
    
    
    

In [29]:
#-----------------------------------------#
testing = np.loadtxt("clean_dataset.txt")
st=time.time()
print(cross_validation(testing))
et=time.time()
print('Runtime: ', et-st)
#-----------------------------------------#

0.975
Runtime:  73.30419969558716


In [17]:
def precision(cm): 
    class_precisions = []
    cols = cm.sum(axis=0) 
    
    for i in range(len(cm)):
        class_precisions.append(cm[i][i]/cols[i]) #calculate precision for each class
    
    return class_precisions

In [18]:
def recall(cm):
    class_recalls = []
    rows = cm.sum(axis=1)
   
    for i in range(len(cm)):
        class_recalls.append(cm[i][i]/rows[i]) #calculate recall for each class
    
    return class_recalls

In [19]:
def f1(recall, precision):
    class_f1s = []
    for i in range(len(recall)):
        class_f1s.append(2*recall[i]*precision[i]/recall[i]+precision[i]) #calculate f1 measure for each class using precision and recall values
    
    return class_f1s
    

In [20]:
def prune(root, validation_set, start):    
    
    if ((root['left']['emitter'] == -1) and (root['right']['emitter'] == -1)): #if both children leaf nodes
        
        old = [root['emitter'], root['value'], root['room'], root['right'], root['left']] #store all parent and children before pruning

        cm, old_accuracy = evaluate(validation_set, start) #obtain accuracy of tree before prune
        
        
        if(root['left']['count'] > root['right']['count']): #choose majority class label using count value
            root['room'] = root['left']['room']
        else:
            root['room'] = root['right']['room']

        #make parent node into single leaf node
        root['left'] = -1 
        root['right'] = -1
        root['emitter'] = -1
        root['value'] = -1
        
        
        cm, new_accuracy = evaluate(validation_set, start) #obtain accuracy of tree after pruning
        
        if new_accuracy < old_accuracy: #if pre-pruned accuracy was better, undo prune
            root['emitter'] = old[0]
            root['value'] = old[1]
            root['room'] = old[2]
            root['right'] = old[3]
            root['left'] = old[4]
        
               
    else:
        
        #traverse tree to find prunable nodes
        if(root['left']['emitter'] != -1): 
            start = prune(root['left'], validation_set, start)
        if(root['right']['emitter'] != -1):
            start = prune(root['right'], validation_set, start)
    
    return start

In [21]:
#===========================================#
#cross validation WITH pruning
#===========================================#

def cross_validation_prune(dataset):

    np.random.shuffle(dataset) #shuffle dataset
    fold_size = int(len(dataset)/10)
    
    #initialise confusion matrices for pre and post pruned trees
    running_cm = np.zeros((4,4))
    running_cm_pr = np.zeros((4,4))
    
    for i in range(10):
        
        #cycle test, training and validation sets
        test = dataset[fold_size*i:fold_size*(i+1)]     
        validation = dataset[fold_size*(i+1)+1:fold_size*(i+2)+1]
        training = np.vstack((dataset[0:fold_size*i],dataset[fold_size*(i+2)+1:]))
        
        #generate decision tree
        root, depth = tree_learning(training, 0)
                
        confusion_matrix, tree_accuracy = evaluate(test, root) #generate confusion matrix for current tree   
        running_cm += confusion_matrix #keep track of confusion matrices to calculate average
              
        pruned_tree = prune(root, validation, root) #generate pruned tree
#       print('Depth after:', findDepth(pruned_tree))
        
        cm_prune, pr_acc = evaluate(test, pruned_tree) #generate confusion matrix for current pruned tree 
        running_cm_pr += cm_prune #keep track of confusion matrices to calculate average for pruned tree
        
#       print(depth, " vs ", findDepth(pruned_tree))
#       print(tree_accuracy, " vs ", pr_acc)

    
    avg_cm = running_cm/10
    avg_cm_pr = running_cm_pr/10
    
#   print("Before prune accuracy:", accuracy(avg_cm))
#   print("After prune accuracy:", accuracy(avg_cm_pr))
    
    return accuracy(avg_cm), accuracy(avg_cm_pr)
    

In [24]:
#===========================================#
#nested cross validation WITH pruning
#===========================================#

def nested_cross_validation_prune(dataset):

    np.random.shuffle(dataset) #shuffle dataset
    fold_size = int(len(dataset)/10)
    
    #initialise confusion matrices for pre and post pruned trees
    running_cm = np.zeros((4,4))
    running_cm_pr = np.zeros((4,4))
    
    for j in range(10):
    
        test = dataset[fold_size*j:fold_size*(j+1)]
        traning_validation_dataset = np.vstack((dataset[0:fold_size*j], dataset[fold_size*(j+1):]))
        
        for i in range(9):
        
            #cycle test, training and validation sets
            
            validation = traning_validation_dataset[fold_size*(i+1)+1:fold_size*(i+2)+1]
            training = np.vstack((traning_validation_dataset[0:fold_size*i],traning_validation_dataset[fold_size*(i+2)+1:]))
        
            #generate decision tree
            root, depth = tree_learning(training, 0)
                
            confusion_matrix, tree_accuracy = evaluate(test, root) #generate confusion matrix for current tree   
            running_cm += confusion_matrix #keep track of confusion matrices to calculate average
                  
            pruned_tree = prune(root, validation, root) #generate pruned tree
            #print('Depth after:', findDepth(pruned_tree))
        
            cm_prune, pr_acc = evaluate(test, pruned_tree) #generate confusion matrix for current pruned tree 
            running_cm_pr += cm_prune #keep track of confusion matrices to calculate average for pruned tree
        
            #print(depth, " vs ", findDepth(pruned_tree))
            #print(tree_accuracy, " vs ", pr_acc)

    
    avg_cm = running_cm/90
    avg_cm_pr = running_cm_pr/90
    
#   print("Before prune accuracy:", accuracy(avg_cm))
#   print("After prune accuracy:", accuracy(avg_cm_pr))
    
    return accuracy(avg_cm), accuracy(avg_cm_pr)
    

In [26]:
#-----------------------------------------#
testing = np.loadtxt("noisy_dataset.txt")
st=time.time()
print(nested_cross_validation_prune(testing))
et=time.time()
print('Runtime: ', et-st)
#-----------------------------------------#

(0.7941666666666666, 0.8166111111111111)
Runtime:  809.2233152389526


In [41]:
def findDepth(node): 
    
    lDepth = 0
    rDepth = 0
    
    if node is None: 
        return 0
  
    else : 

        #find depth for each subtree
        if(node['left']['emitter'] != -1):
            lDepth = findDepth(node['left']) 
        if(node['right']['emitter'] != -1):
            rDepth = findDepth(node['right']) 
  
        #take the larger depth since looking for maximum
        if (lDepth > rDepth): 
            return lDepth+1
        else: 
            return rDepth+1